# 静态web服务器

## 一、静态web服务器

web服务器：web服务器程序。

可以为发出请求的浏览器提供静态文档的程序。一般的网页数据都会改变，但是静态的web服务器提供的内容是静态的、不会改变的。

搭建Python自带的静态web服务器使用

python3 -m http.server 端口号（默认8000）

-m选项：

表示运行包里的模块，执行这个命令需要自己指定静态文件的目录，然后通过浏览器就能访问对应的html文件。

http://localhost:8000/index.html 就可以访问到相应的资源。

## 二、编写自己的静态web服务器

### 1.返回固定页面

In [ ]:
import socket


if __name__ == '__main__':
    tcp_server_socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)

    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

    tcp_server_socket.bind(("", 8000))

    tcp_server_socket.listen(128)

    while True:
        new_socket, ip_port = tcp_server_socket.accept()

        recv_data = new_socket.recv(4096)  # 浏览器的数据上限 4kb左右

        print(recv_data)

        with open('static/index.html', 'r') as file:
            file_data = file.read()
        # 提示：with 不再需要程序员执行关闭代码

        # 只能发回http相应报文封装数据
        response_line = 'HTTP/1.1 200 OK\r\n'
        response_header = 'Server: PWS/1.0\r\n'
        response_body = file_data

        response = response_line + response_header + '\r\n' + response_body

        response = response.encode('utf-8')

        new_socket.send(response)

        new_socket.close()


### 2.返回固定页面及404页面（串行，单一访问）

In [ ]:
import socket


if __name__ == '__main__':
    tcp_server_socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)

    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

    tcp_server_socket.bind(("", 8000))

    tcp_server_socket.listen(128)

    while True:
        new_socket, ip_port = tcp_server_socket.accept()

        recv_data = new_socket.recv(4096)  # 浏览器的数据上限 4kb左右

        recv_data = recv_data.decode('utf-8')

        # 判断的接收数据是否为0
        if len(recv_data) == 0:
            new_socket.close()
            continue

        # 判断请求是不是根目录
        if recv_data.split(' ', 2)[1] == '/':
            path = 'static/index.html'
        else:
            path = 'static/' + recv_data.split(' ', 2)[1]

        try:
            with open(path, 'rb') as file:  # 注意图片必须以二进制进行打开
                file_data = file.read()
            # 提示：with 不再需要程序员执行关闭代码
        except FileNotFoundError:
            with open('static/error.html', 'rb') as file:
                file_data = file.read()
            response_line = 'HTTP/1.1 404 Not Found\r\n'
            response_header = 'Server: PWS/1.0\r\n'
            response_body = file_data
        else:
            response_line = 'HTTP/1.1 200 OK\r\n'
            response_header = 'Server: PWS/1.0\r\n'
            response_body = file_data
        finally:
            response = response_line + response_header + '\r\n'
            response = response.encode('utf-8') + response_body

        new_socket.send(response)

        new_socket.close()


### 3.多任务实现

In [ ]:
import socket
import threading


def show_page(new_socket):
    recv_data = new_socket.recv(4096)  # 浏览器的数据上限 4kb左右

    recv_data = recv_data.decode('utf-8')

    # 判断的接收数据是否为0
    if len(recv_data) == 0:
        new_socket.close()
        return

    # 判断请求是不是根目录
    if recv_data.split(' ', 2)[1] == '/':
        path = 'static/index.html'
    else:
        path = 'static/' + recv_data.split(' ', 2)[1]

    try:
        with open(path, 'rb') as file:  # 注意图片必须以二进制进行打开
            file_data = file.read()
        # 提示：with 不再需要程序员执行关闭代码
    except FileNotFoundError:
        with open('static/error.html', 'rb') as file:
            file_data = file.read()
        response_line = 'HTTP/1.1 404 Not Found\r\n'
        response_header = 'Server: PWS/1.0\r\n'
        response_body = file_data
    else:
        response_line = 'HTTP/1.1 200 OK\r\n'
        response_header = 'Server: PWS/1.0\r\n'
        response_body = file_data
    finally:
        response = response_line + response_header + '\r\n'
        response = response.encode('utf-8') + response_body

    new_socket.send(response)

    new_socket.close()


if __name__ == '__main__':
    tcp_server_socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)

    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

    tcp_server_socket.bind(("", 9000))

    tcp_server_socket.listen(128)

    while True:
        new_socket_out, ip_port = tcp_server_socket.accept()

        sub_thread = threading.Thread(target=show_page, args=(new_socket_out,), daemon=True)

        sub_thread.start()


### 4.面向对象实现

将静态web服务器抽象一个类。

In [ ]:
import socket
import threading


class StaticWeb(object):
    def __init__(self, port=9090):
        self.port = port

        tcp_server_socket = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
        tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

        tcp_server_socket.bind(("", self.port))
        tcp_server_socket.listen(128)

        self.tcp_server_socket = tcp_server_socket

    def run(self):
        while True:
            new_socket_out, ip_port = self.tcp_server_socket.accept()

            sub_thread = threading.Thread(target=StaticWeb.show_page, args=(new_socket_out,), daemon=True)

            sub_thread.start()

    @staticmethod
    def show_page(new_socket: socket.socket):
        data_recv = new_socket.recv(4096).decode('utf-8')

        if len(data_recv) == 0:
            new_socket.close()
            return

        path = data_recv.split(' ', 2)[1]

        if path == '/':
            path = 'static/index.html'
        else:
            path = 'static/' + path

        try:
            file = open(path, 'rb')
        except FileNotFoundError:
            file = open('static/error.html', 'rb')

            file_data = file.read()

            response_line = 'HTTP/1.1 404 Not Found\r\n'
            response_header = 'Server: PWS/1.0\r\n'
            response_body = file_data
        else:
            file_data = file.read()

            response_line = 'HTTP/1.1 200 OK\r\n'
            response_header = 'Server: PWS/1.0\r\n'
            response_body = file_data
        finally:
            response = response_line + response_header + '\r\n'
            response = response.encode('utf-8') + response_body

        new_socket.send(response)

        new_socket.close()


## 三、获取终端命令行参数

命令行中python之后的都是命令行参数。

In [ ]:
import staticweb
from sys import argv


if __name__ == '__main__':
    print(argv)  # 列表中的元素都是字符串
    
    sta_web = None

    if len(argv) == 1:
        sta_web = staticweb.StaticWeb()
    elif len(argv) == 2:
        if argv[1].isdigit():
            sta_web = staticweb.StaticWeb(int(argv[1]))

    sta_web.run()
